In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd

ratings = pd.read_csv('D:\dars\MIR project 2024\IMBD_IR_System\\ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('D:\dars\MIR project 2024\IMBD_IR_System\movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [3]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [5]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [7]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

def genre_recommendations(title, n=10):
    idx = movies[movies['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]


In [9]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
data = pd.merge(ratings, movies, on='movie_id')

In [12]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [13]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [14]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [15]:
# Create User-Genre Matrix
user_genre_matrix = data.pivot_table(index='user_id', columns='genres', aggfunc='size', fill_value=0)
user_genre_matrix = user_genre_matrix.div(user_genre_matrix.sum(axis=1), axis=0)
print(user_genre_matrix.head())

# Calculate User Similarity
user_similarity = cosine_similarity(user_genre_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_genre_matrix.index, columns=user_genre_matrix.index)

def get_top_n_similar_users(user_id, n=5):
    user_sim_scores = user_similarity_df[user_id].sort_values(ascending=False)
    top_n_users = user_sim_scores.iloc[1:n+1].index
    return top_n_users

genres   ["Children's", 'Comedy', 'Drama']  \
user_id                                      
1                                 0.000000   
2                                 0.000000   
3                                 0.000000   
4                                 0.000000   
5                                 0.005051   

genres   ["Children's", 'Comedy', 'Fantasy']  \
user_id                                        
1                                        0.0   
2                                        0.0   
3                                        0.0   
4                                        0.0   
5                                        0.0   

genres   ["Children's", 'Comedy', 'Musical']  \
user_id                                        
1                                   0.018868   
2                                   0.000000   
3                                   0.000000   
4                                   0.000000   
5                                   0.005051   

gen

In [16]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    similar_users = get_top_n_similar_users(user_id, n=n_similar_users)
    similar_users_data = data[data['user_id'].isin(similar_users)]
    
    user_movies = data[data['user_id'] == user_id]['movie_id']
    recommendations = similar_users_data[~similar_users_data['movie_id'].isin(user_movies)]
    
    top_recommendations = recommendations['movie_id'].value_counts().head(n_recommendations).index
    recommended_movies = movies[movies['movie_id'].isin(top_recommendations)]
    
    return recommended_movies

In [17]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [18]:
recommended_movies

,movie_id,title,genres
589,593,"Silence of the Lambs, The (1991)","['Drama', 'Thriller']"
592,596,Pinocchio (1940),"['Animation', ""Children's""]"
1019,1032,Alice in Wonderland (1951),"['Animation', ""Children's"", 'Musical']"
1207,1225,Amadeus (1984),['Drama']
1254,1274,Akira (1988),"['Adventure', 'Animation', 'Sci-Fi', 'Thriller']"
2011,2080,Lady and the Tramp (1955),"['Animation', ""Children's"", 'Comedy', 'Musical..."
2027,2096,Sleeping Beauty (1959),"['Animation', ""Children's"", 'Musical']"
2789,2858,American Beauty (1999),"['Comedy', 'Drama']"
3091,3160,Magnolia (1999),['Drama']
3412,3481,High Fidelity (2000),['Comedy']


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [19]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [20]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# calculate rating matrix
ratings_matrix = np.array(Ratings)


In [22]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_matrix, k=500)
sigma = np.diag(sigma)

In [23]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [24]:
preds =  pd.DataFrame(all_user_predicted_ratings, columns=Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.703333,0.267993,0.227068,0.208574,0.037784,0.211116,-0.071606,-0.089384,0.058296,-0.115528,...,0.036989,-0.030513,0.055480,-0.047841,-0.078905,-0.134897,-0.183411,0.051382,0.063540,0.219349
1,0.186605,-0.384384,0.263561,0.034374,0.244452,-0.113141,0.507855,-0.033870,0.130609,0.877592,...,-0.161415,-0.021699,-0.067089,0.016551,0.006890,-0.335264,-0.349321,-0.043556,-0.030103,0.181778
2,0.370330,0.471853,-0.029554,-0.146391,0.051400,-0.307491,0.244216,-0.060167,0.036427,0.852683,...,0.044072,0.029998,0.040082,0.004715,0.012745,-0.000820,0.204246,0.027996,-0.025839,-0.114927
3,-0.075659,0.114871,-0.080775,-0.050693,0.110614,-0.145656,0.019077,-0.010003,-0.030688,0.045586,...,-0.106984,0.030928,0.034485,-0.075781,-0.034201,-0.036467,0.028688,0.066144,0.027799,-0.072166
4,-0.311929,-0.393238,-0.149400,0.122275,-0.001939,1.964403,0.213568,0.022500,-0.022823,-0.138586,...,0.152565,0.046595,-0.114327,-0.062214,-0.127743,0.435600,0.236111,0.083376,-0.178232,-0.516045


In [25]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == userID]

    # merfe movies data with user_data
    user_full = user_data.merge(movies, how='left', on='movie_id').sort_values(['rating'], ascending=False)
    
    recommendations = movies[~movies['movie_id'].isin(user_full['movie_id'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left', on='movie_id')
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations, :]
    
    return user_full, recommendations

In [26]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

In [27]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [28]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),['Drama']
285,4375,175,5,4374,174,Kids (1995),['Drama']
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)","['Action', 'Drama', 'Romance', 'Thriller']"
65,4375,3809,5,4374,3808,What About Bob? (1991),['Comedy']
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)","['Drama', 'Thriller']"
270,4375,150,5,4374,149,Apollo 13 (1995),['Drama']
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",['Horror']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
60,4375,288,5,4374,287,Natural Born Killers (1994),"['Action', 'Thriller']"
141,4375,47,5,4374,46,Seven (Se7en) (1995),"['Crime', 'Thriller']"


#### Evaluation

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=500)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    user_row_number = user_id - 1 # User ID starts at 1, not 0
    try:
        pred_rating = preds_train.iloc[user_row_number, movie_id]
    except IndexError:
        pred_rating = 0
    return pred_rating

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.5920225489870488


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a fut

### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [30]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [46]:
# Define the dataset    
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.users = torch.tensor(ratings['user_id'].values, dtype=torch.long)
        self.movies = torch.tensor(ratings['movie_id'].values, dtype=torch.long)
        self.ratings = torch.tensor(ratings['rating'].values, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, user, movie):
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)
        x = torch.cat([user_embedded, movie_embedded], dim=-1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# Create the dataset and dataloader
dataset = RatingsDataset(ratings)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function and optimizer
num_users = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()
model = RecommenderNet(num_users + 600, num_movies + 600, embedding_size=50)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    for user, movie, rating in dataloader:
        optimizer.zero_grad()
        outputs = model(user, movie).squeeze()
        loss = criterion(outputs, rating)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    
# Evaluation
model.eval()
def evaluate_model(model, dataset):
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)
    total_loss = 0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for user, movie, rating in dataloader:
            outputs = model(user, movie).squeeze()
            loss = criterion(outputs, rating)
            total_loss += loss.item() * len(rating)
    rmse = (total_loss / len(dataset)) ** 0.5
    return rmse

print(f'RMSE: {evaluate_model(model, dataset)}')

Epoch 1/10, Loss: 1.2023297548294067
Epoch 2/10, Loss: 0.692281186580658
Epoch 3/10, Loss: 0.7261906862258911
Epoch 4/10, Loss: 0.5372017621994019
Epoch 5/10, Loss: 1.224493145942688
Epoch 6/10, Loss: 0.6198557019233704
Epoch 7/10, Loss: 0.4390973448753357
Epoch 8/10, Loss: 0.30238959193229675
Epoch 9/10, Loss: 0.3836527466773987
Epoch 10/10, Loss: 0.8446956872940063
RMSE: 0.8307063609515861


In [47]:
# Example
sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1193, Predicted Rating: 4.421497344970703, Actual Rating: 5.0
Movie ID: 661, Predicted Rating: 3.4709315299987793, Actual Rating: 3.0
Movie ID: 914, Predicted Rating: 4.283843040466309, Actual Rating: 3.0
Movie ID: 3408, Predicted Rating: 4.364636421203613, Actual Rating: 4.0
Movie ID: 2355, Predicted Rating: 4.168822288513184, Actual Rating: 5.0
Movie ID: 1197, Predicted Rating: 4.375242710113525, Actual Rating: 3.0
Movie ID: 1287, Predicted Rating: 4.844799995422363, Actual Rating: 5.0
Movie ID: 2804, Predicted Rating: 4.6092143058776855, Actual Rating: 5.0
Movie ID: 594, Predicted Rating: 4.083475112915039, Actual Rating: 4.0
Movie ID: 919, Predicted Rating: 4.5836358070373535, Actual Rating: 4.0
Movie ID: 595, Predicted Rating: 4.425450801849365, Actual Rating: 5.0
Movie ID: 938, Predicted Rating: 3.99957013130188, Actual Rating: 4.0
Movie ID: 2398, Predicted Rating: 4.131694793701172, Actual Rating: 4.0
Movie ID: 2918, Predicted Rating: 

In [48]:
def predict_unrated_movies(user_id, model, ratings, movies):
    rated_movie_ids = ratings.loc[ratings['user_id'] == user_id, 'movie_id'].unique()
    unrated_movies = movies.loc[~movies['movie_id'].isin(rated_movie_ids)]
    unrated_movie_ids = unrated_movies['movie_id'].values
    user_ids_tensor = torch.full((len(unrated_movie_ids),), user_id, dtype=torch.long)
    movie_ids_tensor = torch.tensor(unrated_movie_ids, dtype=torch.long)
    with torch.no_grad():
        predicted_ratings = model(user_ids_tensor, movie_ids_tensor).numpy()
    unrated_movies = unrated_movies.assign(predicted_rating=predicted_ratings)
    top_recommended_movies = unrated_movies.nlargest(10, 'predicted_rating')

    return top_recommended_movies
# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                                              title  \
315        318                   Shawshank Redemption, The (1994)   
52          53                                    Lamerica (1994)   
2861      2930                           Return with Honor (1998)   
2836      2905                                     Sanjuro (1962)   
359        363  Wonderful, Horrible Life of Leni Riefenstahl, ...   
2254      2323                                 Cruise, The (1998)   
1242      1262                           Great Escape, The (1963)   
3284      3353                         Closer You Get, The (2000)   
1217      1236                                       Trust (1990)   
1178      1196  Star Wars: Episode V - The Empire Strikes Back...   

                                                 genres  predicted_rating  
315                                           ['Drama']          5.115856  
52                                            ['Drama']          4.935105  
2861        

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [59]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

file_path = 'D:\dars\MIR project 2024\IMBD_IR_System\gmm-dataset.csv'
df = pd.read_csv(file_path)

urls = df.iloc[:, 0].values
click_data = df.iloc[:, 1:].values
scaler = StandardScaler()
clicks_normalized = scaler.fit_transform(click_data)

# Fit GMM
gmm = GaussianMixture(n_components=6, random_state=42)  # Adjust n_components as necessary
gmm.fit(clicks_normalized)

# Evaluate model
log_likelihood = gmm.score(clicks_normalized)
bic = gmm.bic(clicks_normalized)
aic = gmm.aic(clicks_normalized)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: -11.521859305847526
BIC: 4123.414084634802
AIC: 3094.3718611695053
